In [78]:
import numpy as np
import pandas as pd
import cv2
from scipy import ndimage
# from skimage.io import imread
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras.activations import relu,linear 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

Data files path settings

In [38]:
# root_path = "/Users/devasenan/Documents/conser-vision/data/"
root_path="D:/Course/DrivenData Project/Dataset/"
# root_path = "C:/Users/devas/Documents/projects/conser-vision/data/"

Read Train features

In [39]:
train_features = pd.read_csv(root_path+"train_features.csv")
train_features.head()

,id,filepath,site
0,ZJ000000,train_features/ZJ000000.jpg,S0120
1,ZJ000001,train_features/ZJ000001.jpg,S0069
2,ZJ000002,train_features/ZJ000002.jpg,S0009
3,ZJ000003,train_features/ZJ000003.jpg,S0008
4,ZJ000004,train_features/ZJ000004.jpg,S0036


In [40]:
train_imgs = []
size = 128
length = 487
train_sites = []

def hist(img):
  img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
  img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
  hist_equalization_result = cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2BGR)
  return hist_equalization_result

# img=cv2.imread(root_path+train_features.iloc[0,1])
# # img=cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
# img=ndimage.gaussian_filter(img,sigma=0.2)
# img=hist(img)
# plt.imshow(img)
# plt.show()



for i in range(length):
    img=cv2.imread(root_path+train_features.iloc[i, 1])
    img = cv2.resize(img, (size, size))
    img=ndimage.gaussian_filter(img,sigma=0.3)
    img=hist(img)
    img = np.array(img).flatten()
    train_imgs.append(img)
    train_sites.append(train_features.iloc[i, 2])
    

In [41]:
train_imgs = np.array(train_imgs)
train_sites = np.array(train_sites)

In [42]:
train_imgs.shape

(487, 49152)

In [43]:
train_imgs = train_imgs/255.0

In [44]:
train_imgs

array([[0.89411765, 0.89411765, 0.89411765, ..., 0.97254902, 0.97254902,
        0.97254902],
       [0.87058824, 0.87058824, 0.87058824, ..., 0.98039216, 0.98039216,
        0.98039216],
       [0.76862745, 0.76862745, 0.76862745, ..., 0.97254902, 0.97254902,
        0.97254902],
       ...,
       [0.56470588, 0.6627451 , 0.52941176, ..., 0.98823529, 0.98823529,
        0.98823529],
       [0.2745098 , 0.2745098 , 0.2745098 , ..., 0.91764706, 0.91764706,
        0.91764706],
       [0.04705882, 0.04705882, 0.04705882, ..., 0.65098039, 0.65098039,
        0.65098039]])

Read train labels

In [45]:
train_lbls = pd.read_csv(root_path+"train_labels.csv")
train_lbls = np.array(train_lbls.iloc[:487, 1:])

In [46]:
train_lbls

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

Read test features

In [47]:
test_features = pd.read_csv(root_path+"test_features.csv")
test_features.head()

,id,filepath,site
0,ZJ016488,test_features/ZJ016488.jpg,S0082
1,ZJ016489,test_features/ZJ016489.jpg,S0040
2,ZJ016490,test_features/ZJ016490.jpg,S0040
3,ZJ016491,test_features/ZJ016491.jpg,S0041
4,ZJ016492,test_features/ZJ016492.jpg,S0040


In [48]:
test_features.shape

(4464, 3)

In [52]:
test_imgs = []
size = 128
length = 464
test_sites = []

for i in range(length):
    img=cv2.imread(root_path+test_features.iloc[i, 1],-1)
    img = cv2.resize(img, (size, size))
    img=ndimage.gaussian_filter(img,sigma=0.3)
#     img=hist(img)
    img=np.array(img).flatten()
    test_imgs.append(img)
    test_sites.append(test_features.iloc[i, 2])


In [59]:
test_imgs = np.array(test_imgs)
test_sites = np.array(test_sites)

In [60]:
test_imgs = test_imgs/255.0

In [61]:
test_imgs.shape

(464,)

In [62]:
train_y = []

for i in train_lbls: 
    train_y.append(list(i).index(1))

train_y

[1,
 6,
 1,
 6,
 5,
 4,
 3,
 0,
 5,
 1,
 5,
 5,
 6,
 2,
 6,
 2,
 5,
 1,
 2,
 6,
 5,
 6,
 0,
 5,
 6,
 7,
 6,
 6,
 6,
 2,
 0,
 6,
 4,
 3,
 6,
 7,
 3,
 0,
 6,
 0,
 3,
 2,
 5,
 0,
 3,
 2,
 5,
 6,
 6,
 0,
 7,
 2,
 4,
 5,
 7,
 6,
 3,
 0,
 6,
 6,
 1,
 2,
 2,
 5,
 2,
 0,
 3,
 0,
 7,
 3,
 2,
 1,
 4,
 3,
 1,
 3,
 7,
 3,
 0,
 1,
 6,
 1,
 2,
 3,
 2,
 3,
 5,
 1,
 0,
 6,
 5,
 2,
 1,
 3,
 0,
 7,
 5,
 5,
 0,
 5,
 2,
 0,
 5,
 6,
 2,
 6,
 5,
 2,
 5,
 4,
 2,
 0,
 7,
 3,
 1,
 3,
 3,
 6,
 7,
 5,
 6,
 6,
 7,
 0,
 0,
 1,
 7,
 0,
 5,
 1,
 7,
 0,
 3,
 3,
 4,
 3,
 3,
 0,
 5,
 3,
 0,
 2,
 2,
 7,
 2,
 0,
 7,
 7,
 7,
 0,
 0,
 3,
 0,
 7,
 1,
 2,
 5,
 0,
 3,
 5,
 7,
 5,
 1,
 1,
 6,
 2,
 5,
 1,
 2,
 7,
 0,
 6,
 2,
 6,
 1,
 5,
 3,
 6,
 3,
 5,
 2,
 2,
 6,
 6,
 7,
 5,
 3,
 0,
 0,
 5,
 1,
 0,
 1,
 2,
 6,
 2,
 2,
 0,
 7,
 6,
 2,
 6,
 3,
 7,
 2,
 7,
 7,
 0,
 1,
 7,
 0,
 0,
 7,
 5,
 6,
 6,
 0,
 7,
 3,
 7,
 4,
 2,
 0,
 7,
 3,
 7,
 7,
 6,
 0,
 5,
 6,
 1,
 2,
 6,
 3,
 5,
 5,
 5,
 0,
 0,
 3,
 2,
 3,
 7,
 5,
 1,
 6,
 0,
 6,
 6,


Feature Extraction

In [18]:
# # FEATURE EXTRACTOR function
# #input shape is (n, x, y, c) number of images, x, y, and channels
# def feature_extractor(dataset):
#     x_train =  dataset
#     image_dataset = pd.DataFrame()
#     for image in range(x_train.shape[0]):  # iterate through each file
#         #print(image)
#         df = pd.DataFrame()  # Temporary data frame to capture information for e
#         #Reset dataframe to blank after each Loop.
#         input_img = x_train[image, :]
#         img = input_img
#         ########
#         #START ADDING DATA TO THE DATAFRAME
#         #Add feature extractors, e.g. edge detection, smoothing, etc.
#         # FEATURE 1 Pixel values
#         #Add pixel values to the data frame
#         pixel_values = img.reshape(-1)
#         df['Pixel_Value'] = pixel_values  # Pixel value itself as a feature
#         #df['Image_Name'] = image #Capture image name as we read multiple ima
       
#         # FEATURE 2 Bunch of Gabor filter responses
#         #Generate Gabor features
#         num = 1  # To count numbers up in order to give Gabor features a Lable {
#         kernels = []
#         for theta in range(2):  # Define number of thetas
#             theta = theta / 4. * np.pi
#             for sigma in (1, 3):  # Sigma with 1 and
#                 lamda = np.pi/4
#                 gamma = 0.5
#                 gabor_label = 'Gabor' + str(num)  # Label Gabor columns as Gabor
#                 print(gabor_label)
#                 ksize = 9
#                 kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma)
#                 kernels.append(kernel)
                                        
#                 fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
#                 filtered_img = fimg.reshape(-1)
#                 # Labels columns as Gabor1, Gabc
#                 df[gabor_label] = filtered_img
#                 print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda)
#                 num += 1


#         image_dataset=image_dataset.append(df)
#     return image_dataset

In [86]:
model =Sequential(
    [
        Conv2D(16,(5,5),input_shape=train_imgs.shape[1:],activation='relu'),
        MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
        MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
        Dropout(0.1),
        Conv2D(32,(5,5),activation='relu'),
        MaxPooling2D(pool_size=(2,2),strides=2,padding='same'),
        Dropout(0.1),
        Conv2D(64,(5,5),activation='relu'),
        Dropout(0.8),
        Flatten(),
        Dense(2,activation='linear')
    ])

ValueError: Input 0 of layer conv2d_21 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 49152]

In [80]:
model_feat=Model(inputs=model.input,outputs=model.get_layer('dense_2').output)
feat_train=model_feat.predict(train_imgs)

NameError: name 'Model' is not defined

In [75]:
# #Extract features from training images
# print(train_imgs)
# image_features = feature_extractor(train_imgs)
# #Reshape to a vector for Random Forest /SVM training
# n_features = image_features.shape[1]
# image_features = np.expand_dims(image_features, axis=0)
# X_for_RF = np.reshape(image_features, (train_imgs.shape[0], -1))

RandomForest

In [20]:
# import tensorflow as tf

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators = 50, random_state = 42)

In [21]:
# n_imgs, img_x, img_y, img_z = train_imgs.shape
# train_imgs_reshaped = train_imgs.reshape(n_imgs, img_x*img_y*img_z)
# train_imgs_reshaped.shape

In [22]:
rf.fit(feat_test,np.argmax(train_y,axis=1))

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
# rf.score(feat_test,np.argmax())

Predictions

In [23]:
# n_imgs, img_x, img_y, img_z = test_imgs.shape
# test_imgs_reshaped = test_imgs.reshape(n_imgs, img_x*img_y*img_z)
# test_imgs_reshaped.shape

In [24]:
#Predict on Test data
#Extract features from test data and reshape, just like training data
from sklearn.metrics import confusion_matrix
from sklearn import metrics
test_features = feature_extractor(test_imgs)
test_features = np.expand_dims(test_features, axis=0)
test_for_RF = np.reshape(test_features, (test_imgs.shape[0], -1))
#Predict on test
test_prediction = model.predict(test_for_RF)
test_prediction


# #Print overall accuracy
# print("Accuracy = ", metrics.accuracy_score(test_labels, test_prediction))
# #Print confusion matrix
# cm = confusion_matrix(test_labels, test_prediction)
# fig, ax = plt.subplots(figsize=(6, 6))  # Sample figsize in inches
# sns.set(font_scale=1.6)
# sns.heatmap(cm, annot-True, ax=ax)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [27]:
test_prediction

array([0, 7, 7, 5, 6, 6, 5, 6, 0, 0, 2, 0, 0, 7, 3, 3, 0, 2, 0, 6, 7, 0,
       7, 5, 5, 0, 7, 0, 7, 0, 7, 6, 7, 2, 7, 6, 5, 5, 6, 3, 6, 6, 0, 7,
       6, 0, 7, 7, 6, 5, 6, 5, 6, 2, 7, 7, 6, 6, 5, 7, 7, 2, 0, 0, 7, 6,
       0, 6, 2, 6, 5, 2, 0, 6, 0, 2, 0, 5, 2, 5, 5, 6, 0, 0, 6, 5, 7, 7,
       7, 5, 0, 0, 6, 6, 6, 2, 0, 5, 0, 2, 7, 5, 0, 2, 0, 0, 2, 5, 0, 1,
       6, 0, 1, 0, 6, 6, 7, 0, 0, 3, 7, 0, 2, 6, 7, 5, 7, 6, 0, 0, 0, 7,
       7, 7, 6, 7, 6, 3, 7, 6, 0, 3, 6, 7, 7, 5, 0, 0, 5, 0, 0, 7, 6, 7,
       7, 0, 0, 6, 6, 7, 0, 7, 2, 2, 5, 6, 6, 1, 0, 6, 5, 0, 0, 3, 2, 2,
       5, 6, 6, 6, 2, 2, 1, 5, 5, 1, 5, 2, 0, 7, 5, 6, 6, 0, 0, 6, 7, 5,
       6, 2, 6, 0, 7, 5, 2, 0, 6, 2, 0, 2, 7, 7, 6, 1, 5, 6, 0, 6, 7, 0,
       5, 0, 6, 6, 0, 6, 3, 7, 5, 5, 5, 0, 0, 6, 6, 7, 2, 2, 0, 0, 7, 6,
       7, 2, 0, 6, 6, 5, 0, 6, 6, 6, 7, 0, 6, 0, 7, 0, 7, 2, 6, 5, 1, 2,
       6, 6, 5, 2, 5, 6, 2, 7, 6, 0, 7, 0, 6, 0, 6, 2, 0, 5, 0, 7, 0, 6,
       5, 0, 3, 6, 5, 0, 5, 0, 5, 1, 0, 2, 5, 0, 0,

In [36]:
submission=pd.read_csv(root_path+"submission_format.csv")
submission = submission.iloc[:, 1:]

In [37]:
test_y = []
k=0
for i in range(submission.shape[0]):
    k=np.argmax(submission.iloc[i,:])
    test_y.append(k)
test_y = np.array(test_y)

In [38]:
test_y

array([3, 6, 2, ..., 2, 2, 6], dtype=int64)

In [39]:
from sklearn.metrics import accuracy_score

accuracy_score(test_y[:464], test_prediction)

0.11206896551724138